In [ ]:
import functools

import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline
import seaborn as sns

import nengo
import nengo.utils.numpy as npext
import nengo_gui.ipython

# Flexible motor oscillators

Goal: Make an oscillator for a syllable
with as few neurons as possible.
Must be able to oscillate at ~2-5 Hz.

In [ ]:
from collections import namedtuple

# Speech action indices
(lab_clos_full, lab_clos_fric,
 api_clos_full, api_clos_fric,
 api_clos_lat, dor_clos_full,
 glott_adduc_phon, glott_abduc_nophon,
 vow_ii, vow_aa, vow_uu) = list(range(11))

# Articulator indices
(tongue_dors_hl, tongue_dors_fb,
 lips_protrusion, lips_constriction,
 ttip_constriction, tdor_constriction) = list(range(6))

# --- Speech Actions to decode from the oscillator
SA = namedtuple('SA', ['onset', 'offset', 'idx'])

def sa_func(name, actions):
    def _fn(x):
        ret = np.zeros(11)
        for action in actions:
            if action.onset <= x <= action.offset:
                ret[action.idx] = 1.0
        return ret
    _fn.__name__ = name
    _fn.actions = actions  # Sneakily store actions
    return _fn

bas_sa = [SA(0.2, 0.4, lab_clos_full),
          SA(0.5, 0.7, api_clos_fric),
          SA(0.2, 0.6, glott_adduc_phon),
          SA(0.5, 0.7, glott_abduc_nophon),
          SA(0.2, 0.6, vow_aa)]
kul_sa = [SA(0.5, 0.7, api_clos_lat),
          SA(0.2, 0.4, dor_clos_full),
          SA(0.5, 0.7, glott_adduc_phon),
          SA(0.2, 0.5, glott_abduc_nophon),
          SA(0.2, 0.6, vow_uu)]
tip_sa = [SA(0.5, 0.7, lab_clos_full),
          SA(0.2, 0.4, api_clos_full),
          SA(0.2, 0.6, glott_adduc_phon),
          SA(0.5, 0.7, glott_abduc_nophon),
          SA(0.2, 0.6, vow_ii)]

bas = sa_func('bas', bas_sa)
kul = sa_func('kul', kul_sa)
tip = sa_func('tip', tip_sa)

In [ ]:
def plot_sa(sa_f):
    x = np.linspace(0, 1, 100)
    out = np.zeros((100, 11))
    for i, xx in enumerate(x):
        out[i] = sa_f(xx)
    plt.plot(x, out)
    plt.ylim(-0.1, 1.1)

plt.figure(figsize=(10, 5))
plt.subplot(3, 1, 1)
plot_sa(bas)
plt.subplot(3, 1, 2)
plot_sa(kul)
plt.subplot(3, 1, 3)
plot_sa(tip)

In [ ]:
from nengo.dists import Choice, Uniform

class Speech(nengo.Network):
    def __init__(self, n_neurons, **netkwargs):
        netkwargs.setdefault('label', 'Speech')
        super(Speech, self).__init__(**netkwargs)
        self.n_neurons = n_neurons
        self.osc = {}  # syllable_f -> oscillator ensemble
        self.inhib = {}  # syllable_f -> inhibitory ensemble

        with self:
            # Make a speech actions EA as a readout
            self.speechactions = nengo.networks.EnsembleArray(n_neurons, 11)
            # Oscillator kick -- this will be an ensemble eventually
            self.osc_kick = nengo.Node(output=lambda t: 0.8 if t < 0.05 else 0.0)
        self.output = self.speechactions.output

    @staticmethod
    def zone(x):
        theta = np.arctan2(x[1], x[0])
        if theta > (3.0 / 4) * np.pi:
            return [0, 0]
        else:
            return x

    @staticmethod
    def radial_f(fn):
        def _fn(x):
            # theta = np.arctan2(x[1], x[0])
            # t = theta / (2 * np.pi) + 0.5
            return fn(np.arctan2(x[1], x[0]) / (2 * np.pi) + 0.5)
        _fn.__name__ = fn.__name__
        return _fn

    def add_syllable(self, syllable_f, tau=0.025, freq=3.3):
        omega = tau * 2 * np.pi * freq
        encoders = [[np.cos(theta), np.sin(theta)]
                    for theta in np.random.uniform(-np.pi, (7.0 / 8) * np.pi, self.n_neurons)]

        with self:
            osc = nengo.Ensemble(self.n_neurons, dimensions=2,
                                 intercepts=Uniform(0.3, 1),
                                 encoders=encoders,
                                 label=syllable_f.__name__)
            nengo.Connection(osc, osc,
                             transform=[[1, -omega], [omega, 1]], function=self.zone, synapse=tau)
            nengo.Connection(self.osc_kick, osc, transform=[[-1], [0]])
            nengo.Connection(osc, self.speechactions.input, function=self.radial_f(syllable_f))
            # By default, oscillator will be inhibited. Inhibit this to remove that.
            inhib = nengo.Ensemble(20, dimensions=1, intercepts=Uniform(-0.4, 0.1), encoders=Choice([[1]]))
            nengo.Connection(inhib.neurons, osc.neurons, transform=-1 * np.ones((self.n_neurons, 20)))

        self.osc[syllable_f] = osc
        self.inhib[syllable_f] = inhib

with nengo.Network() as net:
    sp = Speech(n_neurons=200)
    sp.add_syllable(bas, freq=3.3)
    sp.add_syllable(kul, freq=2.5)
    sp.add_syllable(tip, freq=5.0)
    # disinhibit one to let it through
    disinhibit = nengo.Node(-1)
    nengo.Connection(disinhibit, sp.inhib[bas])

    # Probes
    osc_p = nengo.Probe(sp.osc[bas], synapse=0.01)
    sa_p = nengo.Probe(sp.output, synapse=0.01)

In [ ]:
sim = nengo.Simulator(net)
sim.run(0.4)

In [ ]:
t = sim.trange()
plt.figure()
plt.plot(sim.data[osc_p].T[0], sim.data[osc_p].T[1])
plt.figure()
plt.plot(t, sim.data[sa_p])
plt.xlim(right=t[-1])

In [ ]:
print(sum(ens.n_neurons for ens in net.all_ensembles))

## Speed control

Add in an intermediate population to control speed across all syllables.

In [ ]:
class Speech(nengo.Network):
    def __init__(self, neurons_per_d, **netkwargs):
        netkwargs.setdefault('label', 'Speech')
        super(Speech, self).__init__(**netkwargs)
        self.neurons_per_d = neurons_per_d
        self.osc = {}  # syllable_f -> oscillator ensemble
        self.inhib = {}  # syllable_f -> inhibitory ensemble

        with self:
            # Make a speech actions EA as a readout
            self.speechactions = nengo.networks.EnsembleArray(neurons_per_d, 11)
            # Oscillator kick -- this will be an ensemble eventually
            self.osc_kick = nengo.Node(output=lambda t: 0.8 if t < 0.05 else 0.0)
            # Global speed control
            self.speed = nengo.Ensemble(neurons_per_d, dimensions=1)
        self.output = self.speechactions.output

    @staticmethod
    def zone(x):
        theta = np.arctan2(x[1], x[0])
        if theta > 1.2 * np.pi:
            return 0
        else:
            return x

    @staticmethod
    def radial_f(fn):
        def _fn(x):
            # theta = np.arctan2(x[1], x[0])
            # t = theta / (2 * np.pi) + 0.5
            return fn(np.arctan2(x[1], x[0]) / (2 * np.pi) + 0.5)
        _fn.__name__ = fn.__name__
        return _fn

    def add_syllable(self, syllable_f, tau=0.01, freq=3.3):
        omega = tau * 2 * np.pi * freq
        encoders = [[np.cos(theta), np.sin(theta)]
                    for theta in np.random.uniform(-np.pi, (7.0 / 8) * np.pi, self.neurons_per_d * 2)]

        def feedback(x, w_max=28):
            x0, x1, w = x  # These are the three variables stored in the ensemble
            # w *= -1
            w += 1  # We offset w, so w=0 is normal speed (1.0)
            return Speech.zone(np.array([x0 - w*w_max*tau*x1, x1 + w*w_max*tau*x0]))
        
        with self:
            osc = nengo.Ensemble(self.neurons_per_d * 2, dimensions=2,
                                 intercepts=Uniform(0.3, 1),
                                 encoders=encoders,
                                 label=syllable_f.__name__)
            # Since osc uses special encoders and such, we want to do our control
            # in a separate ensemble. This adds a slight delay, but that's ok.
            ctrl = nengo.Ensemble(self.neurons_per_d * 3, dimensions=3, radius=1.7)
            nengo.Connection(osc, ctrl[:2], synapse=0.005)
            nengo.Connection(self.speed, ctrl[2], synapse=0.005)
            nengo.Connection(ctrl, osc, function=feedback, synapse=tau)
            # Get kick input
            nengo.Connection(self.osc_kick, osc, transform=[[-1], [0]])
            nengo.Connection(osc, self.speechactions.input, function=self.radial_f(syllable_f))
            # By default, oscillator will be inhibited. Inhibit this to remove that.
            inhib = nengo.Ensemble(20, dimensions=1, intercepts=Uniform(-0.4, 0.1), encoders=Choice([[1]]))
            nengo.Connection(inhib.neurons, osc.neurons, transform=-1 * np.ones((osc.n_neurons, 20)))

        self.osc[syllable_f] = osc
        self.inhib[syllable_f] = inhib

with nengo.Network() as net:
    sp = Speech(neurons_per_d=100)
    sp.add_syllable(bas, freq=3.3)
    sp.add_syllable(kul, freq=2.5)
    sp.add_syllable(tip, freq=5.0)
    # disinhibit one to let it through
    disinhibit = nengo.Node(-1)
    nengo.Connection(disinhibit, sp.inhib[bas])

    # Positive = speed up, negative = speed down, -1 = stop
    # Note: if you speed up too much, it'll repeat
    nengo.Connection(nengo.Node(-0.2), sp.speed, synapse=None)

    # Probes
    osc_p = nengo.Probe(sp.osc[bas], synapse=0.01)
    sa_p = nengo.Probe(sp.output, synapse=0.01)
    spd_p = nengo.Probe(sp.speed, synapse=0.01)

In [ ]:
sim = nengo.Simulator(net)
sim.run(0.4)

t = sim.trange()
plt.figure()
plt.plot(sim.data[osc_p].T[0], sim.data[osc_p].T[1])
plt.figure()
plt.plot(t, sim.data[spd_p])
plt.xlim(right=t[-1])
plt.figure()
plt.plot(t, sim.data[sa_p])
plt.xlim(right=t[-1])

In [ ]:
print(sum(ens.n_neurons for ens in net.all_ensembles))